# Enunciado

Gerar um texto, em português, como se fosse um atendente virtual de uma imobiliária.

Através da sintetização do texto, a AV precisa dizer ao cliente que está “ligando” o que ele pode pedir. 

São quatro áreas obrigatórias:
- Vendas
- Aluguel
- Administrativo
- Financeiro

Com o microfone, a pessoa responde uma dessas áreas e através de alguma das funções de reconhecimento de áudio, a área deverá ser identificada e, novamente por sintetização de voz, deverá gerar um áudio indicando que a ligação será transferida para a área correspondente. Por exemplo: “Ótimo, vou transferir você para um de nossos corretores de vendas.”

Deve estar em um loop. E deve haver uma palavra chave pra encerrar o loop. 

Entregar:
- Texto descrevendo o problema e a solução desenvolvida
- Código em Python
- Link para vídeo demonstrando a execução do programa

## Texto descrevendo o problema e a solução desenvolvida


Resposta:

Primeiro definimos as frases que o atendente virtual precisa falar para o atendimento. 

Depois definimos algumas funções auxiliares e por ultimo montamos o fluxo.

Enquanto o usuário não falar a palavra "Tchau" ou "Obrigado(a)" o fluxo ficara em loop esperando por uma das palavras:

- Vendas
- Aluguel
- Administrativo
- Financeiro

Se nada for dito ou a palavra dita não for reconhecida, o fluxo ira responder com uma mensagem de opção inválida.

Utilizamos a bibliotca do Google para fazer a voz sintetizada do atendante. Sobre o usuário, utilizamos a biblioteca PyAudio para gravar o som em conjunto com a biblioteca do Google para transcrever o audio.


## Código em Python

In [5]:
from IPython.display import Audio, display
import numpy as np
import speech_recognition as sr
from gtts import lang, gTTS
from pydub import AudioSegment
from scipy.io.wavfile import write
import pyaudio
import wave
import io
import ffmpeg
from time import sleep
import time
import os
from playsound import playsound

#### Mensagens

In [6]:
HELLO = "Olá, sou o robô falante da imobiliária Casas. Com qual área gostaria de falar? Vendas, Aluguel, Administrativo ou Financeiro"
NOT_UNDERSTAND = "Opção inválida. Por favor, escolha uma das opções: Vendas, Aluguel, Administrativo ou Financeiro" 
SALES = "Ótimo, vou transferir você para um de nossos corretores de vendas"
RENT = "Ótimo, vou transferir você para um de nossos corretores de aluguel"
ADMIN = "Ótimo, vou transferir você para o nosso departamento Administraivo"
FINANCE = "Ótimo, vou transferir você para o nosso departamento Financeiro"
THANKS = "Obrigado por utilizar o robô falante. Até mais!"

#### Funções auxiliares

In [7]:
def start_recording(seconds = 5, filename = "output.wav"):
  chunk = 1024
  sample_format = pyaudio.paInt16
  fs = 44100
  channels = 2

  p = pyaudio.PyAudio()

  print('Recording, speak now...')

  stream = p.open(
    format=sample_format,
    channels=channels,
    rate=fs,
    frames_per_buffer=chunk,
    input=True
  )

  frames = []
  for i in range(0, int(fs / chunk * seconds)):
    data = stream.read(chunk)
    frames.append(data)

  stream.stop_stream()
  stream.close()
  p.terminate()

  print('Finished recording')

  wf = wave.open(filename, 'wb')
  wf.setnchannels(channels)
  wf.setsampwidth(p.get_sample_size(sample_format))
  wf.setframerate(fs)
  wf.writeframes(b''.join(frames))
  wf.close()

def speak(text, lang = 'pt', tld = 'com.br', filename = 'output.mp3'):
  if os.path.exists(filename):
    os.remove(filename) 
  tts = gTTS(text=text, lang=lang, tld=tld)
  tts.save(filename)
  # playsound(filename)

def stt_google(filename, lang = 'pt-BR'):
  recog = sr.Recognizer()
  os.remove(filename)
  try:
    with sr.AudioFile(filename) as source:
      audio = recog.record(source)
      return recog.recognize_google(audio,language=lang)
  except:
    return ''

def exit_loop(transcript):
  if 'obrigad' in transcript.lower() or 'tchau' in transcript.lower():
    return True
  else:
    return False

#### Lógica de funcionamento

In [8]:
speak(HELLO)
sleep(13)
transcript = ''
while (not exit_loop(transcript)):
  start_recording()
  transcript = stt_google('input.wav')
  if 'vend' in transcript.lower():
    speak(SALES)
    sleep(8)
  elif 'administrativ' in transcript.lower():
    speak(ADMIN)
    sleep(8)
  elif 'aluguel' in transcript.lower():
    speak(RENT)
    sleep(8)
  elif 'financeir' in transcript.lower():
    speak(FINANCE)
    sleep(8)
  elif not exit_loop(transcript):
    speak(NOT_UNDERSTAND)
    sleep(13)

speak(THANKS)

KeyboardInterrupt: 

## Link para vídeo demonstrando a execução do programa

Link: